<a href="https://githubtocolab.com/techtribeyt/Deepfake-Maker/edit/main/Deep_Fake_Maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>


# Demo for paper "First Order Motion Model for Image Animation"

This notebook has been modified in order to enable users to specify a video and image in one function call and create a deepfake that gets automatically saved to your Google Drive. Some notes: your video and image have to be in a folder called "deepfake" in your Google Drive. The video and image have to be square shape (will be resized automatically). Also, make sure these files: /content/gdrive/My Drive/deepfake/vox-256.yaml (available on my GitHub), and 
                           /content/gdrive/My Drive/deepfake/vox-cpk.pth.tar (available [here](https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw)) exist in the correct location (deepfake folder).

At this point, your Google Drive folder called "deepfake" should contain any images and videos you want to use, the file "vox-256.yaml" and the file "vox-cpk.pth.tar."

Run all the cells in this Notebook. In the last cell, follow the instructions to make your own calls to the run function. If all arguments are correct, you will see the appropriate deepfake in the deepfake folder in your Drive shortly. To automate this, just run the "run" function multiple times in the same cell, and run the whole cell. Then, you can step away and have multiple deepfakes created while you are away. Also, I recommend using GPU runtime since it will speed up this process.

**Clone repository**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 299 (delta 2), reused 4 (delta 1), pack-reused 293
Receiving objects: 100% (299/299), 72.15 MiB | 25.61 MiB/s, done.
Resolving deltas: 100% (153/153), done.


In [ ]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Load driving video and source image**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
from demo import make_animation
from skimage import img_as_ubyte

def run(vid, img, pref):
  # Change path to your liking or put all of your assets in folder called "deepfake"
  source_image = imageio.imread(f'/content/gdrive/My Drive/deepfake/{img}')
  reader = imageio.get_reader(f'/content/gdrive/My Drive/deepfake/{vid}.mp4')

  image_size = 256
  #Resize image and video to 256x256

  source_image = resize(source_image, (image_size, image_size))[..., :3]

  fps = reader.get_meta_data()['fps']
  driving_video = []
  try:
      for im in reader:
          driving_video.append(im)
  except RuntimeError:
      pass
  reader.close()

  driving_video = [resize(frame, (image_size, image_size))[..., :3] for frame in driving_video]
  predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

  #save resulting video
  imageio.mimsave(f'/content/gdrive/My Drive/deepfake/{vid}_df_{pref}.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

    

**Create a model and load checkpoints**

In [ ]:
# download these files from the Github and put them into the folder
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='/content/gdrive/My Drive/deepfake/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/deepfake/vox-cpk.pth.tar')

**Perform image animation**

In [1]:
# Run like this:
# run("VIDEO_FILE_NAME (without extension)", "SQUARE_IMAGE_NAME.jpg", "PREFIX TO ADD TO FILENAME")
# Ex: run("driving_video", "elon_musk_squared.jpg", "deepfake")